# Instalación de dependencias / Dependencies

In [ ]:
#%pip install opencv-python
#%pip install mediapipe
#%pip install sounddevice
#%pip install git+https://github.com/openai/whisper.git

Programa de visión por computador que analiza los dedos de las manos que el usuario tiene levantados.

En función de qué dedos y de qué mano estén levantados, calcula un número en base 2 que representa esa combinación. La mano izquierda representa los bits menos significativos y la derecha los más significativos, permitiendo representar números del 0 al 1023.

Computer vision program that analyzes which fingers the user has raised.

Based on which fingers and which hand are raised, it calculates a binary number that represents the combination. The left hand represents the least significant bits and the right hand the most significant bits, allowing numbers from 0 to 1023 to be represented.

In [ ]:
import cv2
import mediapipe as mp

# cargar el clasificador preentrenado para detección de rostros
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# inicializar MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.7)

cap = cv2.VideoCapture(0)  # 0 es la webcam principal

def calcular_valor_binario(dedos_levantados, mano_label):
    # Izq (menos significativa) o Der (más significativa)
    # Orden: [Pulgar, Índice, Medio, Anular, Meñique] -> [0, 1, 2, 3, 4]
    pesos = [1 << i for i in range(5)] if mano_label == 'Left' else [1 << (5 + i) for i in range(5)]
    return sum(p for d, p in zip(dedos_levantados, pesos) if d)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)  # espejar la imagen para que sea más intuitivo
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # para detección de rostros

    # detección de rostros
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

    # detección de manos y dedos levantados
    results = hands.process(rgb_frame)
    total_valor = 0
    if results.multi_hand_landmarks:
        for hand_landmarks, hand_info in zip(results.multi_hand_landmarks, results.multi_handedness):
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            dedos = []
            # dedos menos el pulgar
            for tip in [8, 12, 16, 20]:
                dedos.append(int(hand_landmarks.landmark[tip].y < hand_landmarks.landmark[tip - 2].y))
            # pulgar
            dedos.insert(0, int(hand_landmarks.landmark[4].x < hand_landmarks.landmark[3].x))

            # posición de la muñerca
            h, w, _ = frame.shape
            wrist = hand_landmarks.landmark[0]
            x, y = int(wrist.x * w), int(wrist.y * h)

            label = hand_info.classification[0].label  # "Left" o "Right"
            valor = calcular_valor_binario(dedos, label)
            total_valor += valor

            cv2.putText(frame, f"{label} hand: {valor}", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.putText(frame, f"Total: {total_valor}", (10, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)

    cv2.imshow("Webcam - Codificación Binaria con Dedos", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()